In [1]:
import requests
import urllib3
import pandas as pd
from pyhive import presto
import os
import sys
import pymysql
from sqlalchemy import create_engine
from pyhive import presto
import urllib3
import requests
import json
from shapely.geometry import shape, Point
import warnings; warnings.filterwarnings('ignore')
import qgrid
import subprocess

In [2]:
#get credentials (login and password)
os.chdir('C:/Users/maxim.buldakov/Desktop/Private')
ppp = pd.read_csv('ppp.txt')

def query_presto(q):
    presto_session = requests.Session()
    presto_session.verify = 'C:/Users/maxim.buldakov/AppData/Local/Continuum/anaconda3/presto.crt'
    auth={'auth':requests.auth.HTTPBasicAuth(ppp.iloc[0]['---'], ppp.iloc[1]['---'])}
    conn = presto.connect(host="presto-sqlpy.gett.io", port=8443, protocol='https',
                          requests_kwargs=auth, requests_session=presto_session)
    data = pd.read_sql(q, conn)
    conn.close()
    return data

## Queries and processing

In [3]:
start_date = '2020-05-01'
end_date = '2020-05-31'

In [4]:
query_rides_visa = f'''
select
fo.order_gk,
fo.sourceid,
substring(cast(fo.order_gk as varchar(20)),5,length(cast(fo.order_gk as varchar(20)))) as order_gk_str,
fo.riding_user_gk,
fo.order_datetime,
'VISA_Sberbank_20' as financing_type,
fo.date_key as date_key, 
case
when (lower(dct.class_type_desc_eng) like '%vip%' or lower(dct.class_type_desc_eng) like '%luxe%') then 'VIP'
when lower(dct.class_type_desc_eng) like '%comfort%' then 'Comfort'
when lower(dct.class_type_desc_eng) like '%economy%' then 'Economy'
else 'Business' end as class_group,
dct.class_type_desc_eng as class,
fo.customer_total_cost_inc_vat/0.8 as tariff,
fo.customer_total_cost_inc_vat/0.8*0.18 as visa_financing,
fo.customer_total_cost_inc_vat/0.8*0.02 as gett_financing,
fo.customer_total_cost_inc_vat as client_price,
fo.paid_with_prepaid

from emilia_gettdwh.dwh_fact_orders_v fo
join emilia_gettdwh.dwh_dim_class_types_v dct on dct.class_type_key = fo.class_type_key

where
fo.order_status_key in (7)
and fo.date_key between DATE '{start_date}' and DATE '{end_date}'
and fo.country_key = 2
and fo.payment_type_key = 3
and lower(dct.class_type_desc) like '%visa%'
and lower(dct.class_type_desc) like '%sber20%'
and lower(dct.class_type_desc) not like '%alfa%'
and lower(dct.class_type_desc) not like '%open20%'
'''

rides_visa = query_presto(query_rides_visa)

In [5]:
print(rides_visa.shape)
print(rides_visa['order_gk_str'].unique().shape)

(1364, 14)
(1364,)


In [6]:
print(rides_visa[['client_price', 'order_gk']].info())
print('----------')
print('rows with zero tariff:', rides_visa['tariff'][rides_visa['tariff'].astype('float64') == 0].count())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 2 columns):
client_price    1362 non-null object
order_gk        1364 non-null int64
dtypes: int64(1), object(1)
memory usage: 21.4+ KB
None
----------
rows with zero tariff: 0


## If many NaN figures do next

In [49]:
query_add = f'''
select o.id
,o.cost_total_for_company_inc_vat

from "gt-ru".gettaxi_ru_production.orders o

where o.status_id = 7
and o.created_at between (date '{start_date}' - interval '1' day) and (date '{end_date}' + interval '1' day)
'''

summ = query_presto(query_add)
print('summ.shape:', summ.shape)

summ.shape: (2731722, 2)


In [50]:
q = pd.merge(rides_visa, summ, left_on = 'sourceid', right_on = 'id', how ='left')
q = q.drop(columns = ['sourceid', 'id'])
change = ['tariff','visa_financing','gett_financing','client_price','cost_total_for_company_inc_vat']

for i in range(0, len(change)):
    q[change] = q[change].astype('float64')
    
#calculation
q['tariff2'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']/0.8, axis = 1)
q['visa_financing2'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']/0.8*0.18, axis = 1)
q['gett_financing2'] = q.apply(lambda x: x['cost_total_for_company_inc_vat']/0.8*0.02, axis = 1)

#change if NaN figures in main base
q['tariff'] = q.apply(lambda x: x['tariff2'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['tariff'], axis = 1)
q['visa_financing'] = q.apply(lambda x: x['visa_financing2'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['visa_financing'], axis = 1)
q['gett_financing'] = q.apply(lambda x: x['gett_financing2'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['gett_financing'], axis = 1)
q['client_price'] = q.apply(lambda x: x['cost_total_for_company_inc_vat'] if (x['client_price'] == 0 or pd.isnull(x['client_price']))
                      else x['client_price'], axis = 1)
q = q.drop(columns = ['tariff2', 'visa_financing2', 'gett_financing2', 'cost_total_for_company_inc_vat'])
rides_visa = q.copy()

In [51]:
rides_visa[['client_price', 'order_gk']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2291 entries, 0 to 2290
Data columns (total 2 columns):
client_price    2291 non-null float64
order_gk        2291 non-null int64
dtypes: float64(1), int64(1)
memory usage: 53.7 KB


## Continue

In [7]:
if 'sourceid' in rides_visa.columns:
    rides_visa.drop(columns = 'sourceid', inplace = True)
    print('column sourceid deleted (no NaN figures)')
else:
    print('column sourceid not found (NaN figures part ran)')

column sourceid deleted (no NaN figures)


In [8]:
rides_visa['date_key_time'] = pd.to_datetime(rides_visa['date_key'])
rides_visa['date_key'] = rides_visa['date_key_time'].dt.date
uninque_orders = tuple(list(rides_visa['order_gk']))

In [9]:
query_ftrs = f'''
with all_rides as
(select
fo.order_gk,
du.registration_date_key,
fo.date_key,
fo.ride_start_datetime as visa_ride_start_datetime,
fr.ride_start_datetime as previous_rides_start_datetime,
row_number() over (partition by fo.order_gk order by fo.order_gk, fr.ride_start_datetime ) is_ftr

from emilia_gettdwh.dwh_fact_orders_v fo
join emilia_gettdwh.dwh_fact_rides_v fr on fr.riding_user_gk = fo.riding_user_gk and fr.ride_start_datetime <= fo.ride_start_datetime
join emilia_gettdwh.dwh_dim_users_v du on du.user_gk = fo.riding_user_gk
join emilia_gettdwh.dwh_dim_class_types_v dct on dct.class_type_key = fr.class_type_key

where
fo.order_gk in {uninque_orders})

select
ar.order_gk,
case when ar.is_ftr = 1 and DATE_DIFF('day', ar.registration_date_key, ar.date_key) between 0 and 90 then 1 else 0 end as is_ftr
from all_rides ar
where ar.visa_ride_start_datetime = ar.previous_rides_start_datetime
'''

ftrs_pd = query_presto(query_ftrs)
print(ftrs_pd.shape)

(1364, 2)


In [10]:
rides_visa = pd.merge(rides_visa, ftrs_pd, left_on = 'order_gk', right_on = 'order_gk', how ='left')
print(rides_visa.shape)

(1364, 15)


In [11]:
query_tranc = f'''
select
fct.order_gk
,dcc.last4
,dcc.bin_number
,tx_type

from dwh.dim_credit_cards_v dcc
join dwh.fact_ccs_transactions_v fct on fct.credit_card_gk = dcc.credit_card_gk

where fct.order_gk in {uninque_orders}
and fct.status = 'completed' 
and tx_type in ('charge', 'prepaid', 'settle')
and fct.exception_text is null
'''

df_trans = query_presto(query_tranc)
print(df_trans.shape)
print(df_trans['order_gk'].unique().shape)

(1373, 4)
(1353,)


In [12]:
df_trans['bin_number'] = list(map(lambda x: int(x), df_trans['bin_number']))

In [13]:
df = pd.merge(rides_visa, df_trans, left_on = 'order_gk', right_on = 'order_gk', how = 'left')
print(df.shape)

(1384, 18)


In [14]:
df = df.dropna(subset = ['bin_number'])
df['bin_number'] = list(map(lambda x: int(x),  df['bin_number']))
print(df.shape)
print(df['order_gk_str'].unique().shape)

(1373, 18)
(1353,)


In [15]:
tt = df.copy()

In [16]:
os.chdir('C:/Users/maxim.buldakov/Desktop/Private/VISA')

df_bin_conditions = pd.read_excel('visa_sber_mass-classes_v.xlsx')
df_bin_conditions[:5]

,BIN,CARD_TYPE,start_date,end_date
0,427400,VISA BUSINESS,2018-01-10,2020-12-31
1,427402,VISA BUSINESS,2018-01-10,2020-12-31
2,427403,VISA BUSINESS,2018-01-10,2020-12-31
3,427404,VISA BUSINESS,2018-01-10,2020-12-31
4,427406,VISA BUSINESS,2018-01-10,2020-12-31


In [17]:
df_fin = pd.merge(df, df_bin_conditions, left_on = 'bin_number', right_on = 'BIN', how = 'inner')

In [18]:
def if_date_comp(st,en, r):
    if r <= en.date() and  r >= st.date():
        v = 1
    else:
        v = 0
    return v

df_fin['is_in_group'] = list(map(lambda st,en,r: if_date_comp(st,en,r), df_fin['start_date'] , df_fin['end_date'] , df_fin['date_key']   ) )

In [19]:
df_fin.date_key = pd.to_datetime(df_fin.date_key)
df_fin["start_date"] = pd.to_datetime(df_fin.start_date).apply(lambda x: x.date())
df_fin["end_date"] = pd.to_datetime(df_fin.end_date).apply(lambda x: x.date())
df_fin["start_date"] = pd.to_datetime(df_fin.start_date).dt.date
df_fin["end_date"] = pd.to_datetime(df_fin.end_date).dt.date
df_fin.date_key = pd.to_datetime(df_fin.date_key).dt.date

In [20]:
df_fin = df_fin[df_fin['is_in_group'] == 1]

In [21]:
z = pd.concat([df_fin[df_fin.financing_type == 'VISA_Sberbank_20']
               , df_fin[df_fin.financing_type != 'VISA_Sberbank_20']], axis = 0).drop_duplicates() #.to_excel('visa_report_august_to31.xlsx')

In [22]:
x = z.copy()

In [23]:
z = pd.concat([df_fin[df_fin.financing_type == 'VISA_Sberbank_20']
               , df_fin[df_fin.financing_type != 'VISA_Sberbank_20']], axis = 0)
z.shape

(1363, 23)

In [24]:
z = pd.concat([df_fin[df_fin.financing_type == 'VISA_Sberbank_20']
               , df_fin[df_fin.financing_type != 'VISA_Sberbank_20']], axis = 0).drop_duplicates()
z.shape

(1363, 23)

## Google sheets

In [25]:
## В ЭТОМ БЛОКЕ ПОДКЛЮЧАЕМ НЕОБХОДИМЫЕ БИБЛИОТЕКИ

# Стандартные библиотеки для работы с DataFrame - так большие данные хранит Python
import pandas as pd

# Библиотеки для работы с временем
import datetime
import time

#Библиотеки для подключения к GoogleSheets
import gspread 
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd

In [26]:
import webbrowser

site1 =  "https://docs.google.com/spreadsheets/d/1e5GjaQvmqs0xrhjJVc-FpN2TIpeQGpWw1WYmjLyW63s/edit#gid=2106537651"
webbrowser.open(site1, new = 2)

True

In [27]:
## В ЭТОМ БЛОКЕ ПОДКЛЮЧАЕМСЯ К GOOGLE SHEETS ТАБЛИЦЕ

# Стандартные строчки
scope = ['https://spreadsheets.google.com/feeds']

# Название файла с данными от клиента
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/maxim.buldakov/Desktop/Private/gapi-prj1.json', scope)
client = gspread.authorize(creds)

# Ссылка на таблицу, куда подключаемся 
table = client.open_by_url(site1)

In [28]:
# Подключаемся к конкретному листу в таблице
ws = table.worksheet('report')
# Очищаем этот лист
ws.clear()

{'spreadsheetId': '1e5GjaQvmqs0xrhjJVc-FpN2TIpeQGpWw1WYmjLyW63s',
 'clearedRange': 'report!A1:Z2828'}

In [29]:
#add part
x1 = x.copy()
x1 = x1.drop(['order_gk', 'riding_user_gk', 'date_key_time', 'start_date', 'end_date', 'is_in_group','order_datetime'], axis = 1)
x1['paid_with_prepaid'] = x1['paid_with_prepaid'].astype('float64')
x1['paid_with_prepaid'] = x1['paid_with_prepaid'].fillna(0)
x1['paid_with_prepaid'] = x1['paid_with_prepaid'].apply(lambda row: 1 if row > 0 else 0)
x1 = x1.sort_values(by = ['order_gk_str', 'tx_type'])
gd.set_with_dataframe(ws, x1, row=1, col=1)

In [30]:
dates = {'From': [start_date], 'To': [end_date]}
dates = pd.DataFrame(data=dates)

In [31]:
summary = table.worksheet('summary')
#summary.clear()

In [32]:
gd.set_with_dataframe(summary, dates, row=1, col=1)

In [33]:
# Подключаемся к конкретному листу в таблице
ws1 = table.worksheet('raw')
# Очищаем этот лист
ws1.clear()

{'spreadsheetId': '1e5GjaQvmqs0xrhjJVc-FpN2TIpeQGpWw1WYmjLyW63s',
 'clearedRange': 'raw!A1:Z2979'}

In [34]:
gd.set_with_dataframe(ws1, tt, row=1, col=1)

In [35]:
#end of script